<a href="https://colab.research.google.com/github/MariaPetrovskaya/Data-Science-demo/blob/main/Matrix_for_Data_Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import LinearRegression
from numpy.linalg import inv
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Загрузка всех библиотек

In [ ]:
df = pd.read_csv('/datasets/insurance.csv')
data  = df.copy()
#
print('загрузка и знакомство с данными')
print (data.shape)
print (data.info())
print (data.head())

загрузка и знакомство с данными
(5000, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None
   Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
0    1     41.0   49600.0            1                  0
1    0     46.0   38000.0            1                  1
2    0     29.0   21000.0            0                  0
3    0     21.0   41700.0            2                  0
4    1     28.0   26100.0            0                  0


Данные загружены, все в чиловых форматах, пропусков нет, размер 5000х5. Столбцы "Пол" и "Страховые выплаты" - по сути булиевы, но хранятся как int, что удобно для дальнейших расчетов

In [ ]:
display(data.corr())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.002074,0.014910,-0.008991,0.010140
Возраст,0.002074,1.000000,-0.019093,-0.006692,0.651030
Зарплата,0.014910,-0.019093,1.000000,-0.030296,-0.014963
Члены семьи,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
Страховые выплаты,0.010140,0.651030,-0.014963,-0.036290,1.000000


Посчитаны корреляции столбцов данных между собой. Наблюдается корреляция между "Возраст" и "Страховые выплаты". "Страховые выплаты" - в дальнейшем таргет. Вероятно, "Возраст" - основной действующий фактор, с другими столбцами не коррелирует. Убирать этот столбец нельзя

## Умножение матриц

Сначала проверим саму по себе применимость матриц для шифровки и дешифровки датасетов

In [ ]:
A = np.random.normal(size=(5, 5))
Ainv = inv(np.matrix(A))
print(A)
print(Ainv)

[[-0.39526637  0.49420434  0.8852591   0.88915247  0.9283346 ]
 [ 1.26196554  1.02051653 -1.67287637 -0.48559911  0.51356171]
 [-0.48328232 -0.07827841  0.86802506 -1.12043564 -0.68514389]
 [ 1.06937326 -3.06802569  0.92969276  0.10337996  0.50362091]
 [-1.01413656 -0.33117213 -2.65893565  0.13118759  1.28583909]]
[[-0.06608734  0.3520712  -0.22329939  0.1409937  -0.26710863]
 [ 0.20535989  0.1839457   0.05014491 -0.22126052 -0.10835124]
 [ 0.35987337  0.0227933   0.26422634  0.07584685 -0.15783753]
 [-0.17338399 -0.3995729  -0.88449201 -0.12335278 -0.13821136]
 [ 0.76262564  0.41295283  0.47342295  0.22364058  0.2268435 ]]


Сгенерирована случайная матрица и обратная к ней

In [ ]:
matrix = data.values
print(matrix)
print('Размер:', matrix.shape)

[[1.00e+00 4.10e+01 4.96e+04 1.00e+00 0.00e+00]
 [0.00e+00 4.60e+01 3.80e+04 1.00e+00 1.00e+00]
 [0.00e+00 2.90e+01 2.10e+04 0.00e+00 0.00e+00]
 ...
 [0.00e+00 2.00e+01 3.39e+04 2.00e+00 0.00e+00]
 [1.00e+00 2.20e+01 3.27e+04 3.00e+00 0.00e+00]
 [1.00e+00 2.80e+01 4.06e+04 1.00e+00 0.00e+00]]
Размер: (5000, 5)


создана матрица matrix из исходного датасета

In [ ]:
crypted = matrix@A
print(crypted)
print('Размер:', crypted.shape)

[[-23918.38814493  -3843.34188286  42987.26990096 -55592.52471216
  -33960.64913752]
 [-18306.62232993  -2931.03509647  32906.27065076 -42598.65726334
  -26010.05465859]
 [-10112.33162068  -1614.25167429  18180.01280475 -23543.23078758
  -14373.12847626]
 ...
 [-16355.8924314   -2639.36389038  29394.45132668 -37992.27337526
  -23215.09951742]
 [-15772.75561507  -2545.96258416  28351.29045612 -36647.72927458
  -22390.46776615]
 [-19585.25285923  -3152.10288904  35196.79177114 -45502.29117513
  -27801.03039725]]
Размер: (5000, 5)


путем умножения создана зашифрованная матрица

In [ ]:
uncrypted = crypted@Ainv
print(uncrypted)
print('Размер:', uncrypted.shape)

[[ 1.00000000e+00  4.10000000e+01  4.96000000e+04  1.00000000e+00
  -3.67953614e-13]
 [ 9.82338202e-13  4.60000000e+01  3.80000000e+04  1.00000000e+00
   1.00000000e+00]
 [-1.37460182e-12  2.90000000e+01  2.10000000e+04  1.69346955e-13
   1.14689720e-13]
 ...
 [-2.16976319e-12  2.00000000e+01  3.39000000e+04  2.00000000e+00
   2.57621779e-13]
 [ 1.00000000e+00  2.20000000e+01  3.27000000e+04  3.00000000e+00
   5.79264758e-13]
 [ 1.00000000e+00  2.80000000e+01  4.06000000e+04  1.00000000e+00
  -4.40390274e-13]]
Размер: (5000, 5)


проверка того, как работает расшифровка (умножение зашифрованной на обратную матрицу) - вижу сходство и первоначальными данными

**Результат** Проверена примененимость матрицы и обратной матрицы для шифровки и дешифровки данных

**Теоретическая часть**

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** 
<br>
Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?
<br>
**Ответ:** не изменится.
<br>
<br>
**Доказательство:**
<br>
<br>
<br>


<br>
Заменим матрицу $X$ на матрицу $Z$
<br>
$$Z = XP$$
<br>
где $P$ - обратимая матрица с некими значениями, на которую может быть умножена матрица $X$
<br>
вычислим, чему будет равено предсказание и вектор весов:
<br>
    $$a_1 = Zw_1$$
<br>
    $$ w_1 = (Z^T Z)^{-1} Z^T y $$
<br> 
далее сопоставим уравнения и получим:
<br>
$$a_1 = Z (Z^T Z)^{-1} Z^T y $$
<br>
заменим $Z = XP$ в уравнении:
<br>
$$ a_1 = XP ((XP)^T (XP))^{-1} (XP)^T y $$
<br>
применим следующее свойство обратной матрицы:
<br>
    $$ (AB)^{-1} = B^{-1} A^{-1} $$
<br>
Раскроем $((XP)^T (XP))^{-1}$ в два шага:
<br>
   $$a_1 = XP ((XP)^T (XP))^{-1} (XP)^T y  = XP(XP)^{-1}((XP)^T)^{-1} (XP)^T y = XPP^{-1}X^{-1}((XP)^T)^{-1} (XP)^T y $$
   <br>
Умножение приведет к $PP^{-1} = E$
<br>
Для следующего этапа воспользуемся свойством транспорнированной матрицы:
<br>
    $$ (AB)^T = B^T A^T$$
<br>    
Умножение на единичную матрицу не меняет матрицу. 
<br>
Раскроем $((XP)^T)^{-1} (XP)^T$ в три шага:
<br>
 $$a_1 = XEX^{-1}((XP)^T)^{-1} (XP)^T y = XX^{-1}(P^T X^T)^{-1} P^T X^T y = XX^{-1} (X^T)^{-1} (P^T)^{-1}  P^T X^T y $$
 <br>
 Умножение приводит к $(P^T)^{-1}  P^T = E$
 <br>
 тогда:
 <br>

 $$ a_1 = XX^{-1} (X^T)^{-1} E X^T y = X(X^T X)^{-1} X^T y = Xw = a $$
<br> 
Как видно, значение предсказания $a$ не меняется, если умножать матрицу признаков на обратимую матрицу


## Алгоритм преобразования

**Этапы алгоритма:**
<br>
 Генерация матрицы $P$
<br>
 Проверка матрицы на обратимость(генерация обратной матрицы)
 <br>
 Получение матрицы преобразованных признаков $Z = X/P$.
 <br>
 Применение алгоритма на преобразованных признаках $Z$.

 <br>
 Матрица $P$ должна иметь необходимую размерность $(nxn)$, где n - количество признаков для регрессии
 <br>
 Таким образом матрица $Z$ будет иметь туже размерность, что и матрица $X$. 
 <br>
 Обратная матрица $P$ возможна только для квадратных невырожденных матриц (определитель которых не равен нулю)
 <br>
 Проведем проверку но обратимость или применим функцию генерации обратимой матрицы
    

In [ ]:
#def get_rand_matrix():
    #det = 0
    #while det == 0:
        #matrix = np.random.normal(size=(data.shape[1], data.shape[1]))
        #det = np.linalg.det(matrix)
    #return matrix

## Проверка алгоритма

Применим модель линейной регрессии первоначально к данным без шифрования

In [ ]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']
print(features.shape)
print(target.shape)

(5000, 4)
(5000,)


выделили таргет и признаки

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)
print(features_train.shape)
print(features_test.shape)
print(target_train.shape)
print(target_test.shape)

(3750, 4)
(1250, 4)
(3750,)
(1250,)


разбиваем не шифрованные данные на обучающую и тестовую выборки

In [ ]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)# < напишите код здесь >
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print("R2",r2_score(target, predictions))


R2 0.42494550286668


К данным до шифрования применена модель линейной регрессии. Параметры w и w0, w вычисляется по формуле минимизации MSE $$w = \arg\min_w MSE(Xw, y)$$

In [ ]:
model = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller),("linear_regression", model)])
pipeline.fit(features_train, target_train)
R2_scaled = r2_score(target_test, pipeline.predict(features_test))
print("R2_scaled =", R2_scaled)


R2_scaled = 0.4352275712702667


Для сравнения применен StandardScaler (масштабирование), показатель незначительно вырос

In [ ]:
B = np.random.normal(size=(4, 4))
Binv = inv(np.matrix(B))
print(B)
print(Binv)

[[ 0.72858609  0.16036753 -0.82621791 -0.73681412]
 [ 0.66114662 -0.93174723 -0.72418526 -1.39834946]
 [ 1.1494969  -0.75731065  0.70078185  1.39024734]
 [ 0.734172   -1.22085534  0.6255142  -0.46627449]]
[[ 1.18961711 -0.70310939  0.12885067  0.61294421]
 [ 1.26947108 -1.3294852  -0.44941547  0.64108596]
 [ 0.6278118  -1.36368508 -0.54814397  1.46324109]
 [-0.60855094  0.54453225  0.64425139 -0.89515657]]


для шифровки признаков генеририруем новую матрицу размера 4х4 и на всякий случай обратную матрицу.

In [ ]:
features_matrix = features.values
print(features_matrix)
print('Размер:', features_matrix.shape)

[[1.00e+00 4.10e+01 4.96e+04 1.00e+00]
 [0.00e+00 4.60e+01 3.80e+04 1.00e+00]
 [0.00e+00 2.90e+01 2.10e+04 0.00e+00]
 ...
 [0.00e+00 2.00e+01 3.39e+04 2.00e+00]
 [1.00e+00 2.20e+01 3.27e+04 3.00e+00]
 [1.00e+00 2.80e+01 4.06e+04 1.00e+00]]
Размер: (5000, 4)


преобразуем признаки в матрицу

In [ ]:
features_crypted = features_matrix@B
print(features_crypted)
print('Размер:', features_crypted.shape)

[[ 57043.61600745 -37601.87038037  34728.88743822  68897.7326709 ]
 [ 43712.0291149  -28821.88594026  26597.0232751   52764.6085882 ]
 [ 24158.60815107 -15930.54432649  14695.417468    29154.64201551]
 ...
 [ 38982.63618502 -25693.90770131  23743.27202286  47100.48530382]
 [ 37606.02495638 -24788.05890318  22900.68472917  45428.18870771]
 [ 46689.54900178 -30773.96181345  28431.26520065  56403.68514966]]
Размер: (5000, 4)


In [ ]:
features_crypted_train, features_crypted_test, target_train, target_test = train_test_split(
    features_crypted, target, test_size=0.25, random_state=12345)
print(features_crypted_train.shape)
print(features_crypted_test.shape)
print(target_train.shape)
print(target_test.shape)

(3750, 4)
(1250, 4)
(3750,)
(1250,)


In [ ]:

    
model = LinearRegression()
model.fit(features_crypted, target)
predictions_crypted = model.predict(features_crypted)
print("R2_crypted =", r2_score(target, predictions_crypted))

R2_crypted = 0.42494550286668475


показатель r2 такой же, как и до шифрования без StandardScaler

In [ ]:
model = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller),("linear_regression", model)])
pipeline.fit(features_crypted_train, target_train)
R2_crypted_scaled = r2_score(target_test, pipeline.predict(features_crypted_test))
print("R2_crypted_scaled =", R2_crypted_scaled)

R2_crypted_scaled = 0.435227571303371


применен StandardScaler (масштабирование), показатель незначительно вырос

In [ ]:
print('Сравнение метрик R2 при различных преобразованиях данных')
print()
print("данные в первоночальном виде")
print("R2",r2_score(target, predictions))
print()
print("данные до шифрования с использованием StandardScaler")
print("R2_scaled =", R2_scaled)
print()
print("данные после шифрования без испльзования StandardScaler")
print("R2_crypted =", r2_score(target, predictions_crypted))
print()
print("данные после шифрования с использованием StandardScaler")
print("R2_crypted_scaled =", R2_crypted_scaled)

Сравнение метрик R2 при различных преобразованиях данных

данные в первоночальном виде
R2 0.42494550286668

данные до шифрования с использованием StandardScaler
R2_scaled = 0.4352275712702667

данные после шифрования без испльзования StandardScaler
R2_crypted = 0.42494550286668475

данные после шифрования с использованием StandardScaler
R2_crypted_scaled = 0.435227571303371


**Выводы** Показатели R2 схожи до шифрования и после, StandardScaler показывает аналогичное влияние в обоих случаях

**Общие выводы**
<br>
Данные загружены и изучены.
<br>
Проверена применимость матриц для шифровки данных в датасетах.
<br>
Качество модели линейной регрессии одинаковое для шифрованных и не шифрованных данных.
<br>
Создан алгоритм преобразования данных.
<br>
Применен алгоритм преобразования данных, оценивалась метрика R2 для данных до преобразования и после.
<br>
Результаты демонстрируют применимость и удобство матриц для шифровки данных в датасете.